
# Baseline feature transformation

The simulated dataset generated in the previous section is simple. It only contains the essential features that characterize a payment card transaction. These are: a unique identifier for the transaction, the date and time of the transaction, the transaction amount, a unique identifier for the customer, a unique number for the merchant, and a binary variable that labels the transaction as legitimate or fraudulent (0 for legitimate or 1 for fraudulent).






In this section, we will implement three types of feature transformation that are known to be relevant for payment card fraud detection.

The first type of transformation involves the date/time variable, and consists in creating binary features that characterize potentially relevant periods. We will create two such features. The first one will characterize whether a transaction occurs during a weekday or during the weekend. The second will characterize whether a transaction occurs during the day or the night. These features can be useful since it has been observed in real-world datasets that fraudulent patterns differ between weekdays and weekends, and between the day and night.  

The second type of transformation involves the customer ID and consists in creating features that characterize the customer spending behaviors. We will follow the RFM (Recency, Frequency, Monetary value) framework proposed in {cite}`VANVLASSELAER201538`, and keep track of the average spending amount and number of transactions for each customer and for three window sizes. This will lead to the creation of six new features.

The third type of transformation involves the terminal ID and consists in creating new features that characterize the 'risk' associated with the terminal. The risk will be defined as the average number of frauds that were observed on the terminal for three window sizes. This will lead to the creation of three new features. 

The table below summarizes the types of transformation that will be performed and the new features that will be created. 

|Original feature name|Original feature type|Transformation|Number of new features|New feature(s) type|
|---|---|---|---|---|
|TX\_DATE\_TIME | Panda timestamp |0 if transaction during a weekday, 1 if transaction during a weekend. The new feature is called TX_DURING_WEEKEND.|1|Integer (0/1)|
|TX\_DATE\_TIME | Panda timestamp |0 if transaction between 6am and 0pm, 1 if transaction between 0pm and 6am. The new feature is called TX_DURING_NIGHT.|1|Integer (0/1)|
|CUSTOMER\_ID | Categorical variable |Number of transactions by the customer in the last n day(s), for n in {1,7,30}. The new features are called CUSTOMER_ID_NB_TX_nDAY_WINDOW.|3|Integer|
|CUSTOMER\_ID | Categorical variable |Average spending amount in the last n day(s), for n in {1,7,30}. The new features are called CUSTOMER_ID_AVG_AMOUNT_nDAY_WINDOW.|3|Real|
|TERMINAL\_ID | Categorical variable |Number of transactions on the terminal in the last n+d day(s), for n in {1,7,30} and d=7. The parameter d is called delay and will be discussed later in this notebook. The new features are called TERMINAL_ID_NB_TX_nDAY_WINDOW.|3|Integer|
|TERMINAL\_ID | Categorical variable |Average number of frauds on the terminal in the last n+d day(s), for n in {1,7,30} and d=7. The parameter d is called delay and will be discussed later in this notebook. The new features are called TERMINAL_ID_RISK_nDAY_WINDOW.|3|Real|

The following sections provide the implementation for each of these three transformations. After the transformations, a set of 14 new features will be created. Note that some of the features are the result of aggregation functions over the values of other features or conditions (same customer, given time window). These features are often referred to as *aggregated features*.

In [39]:
# Initialization: Load shared functions and simulated data 

%run shared_functions.py

## Loading of dataset

 


In [40]:
DIR_INPUT='../data/raw/transaction/' 

BEGIN_DATE = "2024-06-01"
END_DATE = "2024-12-31"

print("Load  files")
%time transactions_df=read_from_files(DIR_INPUT, BEGIN_DATE, END_DATE)
print("{0} transactions loaded, containing {1} fraudulent transactions".format(len(transactions_df),transactions_df.TX_FRAUD.sum()))


Load  files
CPU times: user 4.99 s, sys: 2 s, total: 6.99 s
Wall time: 21.2 s
2051331 transactions loaded, containing 17274 fraudulent transactions


In [41]:
transactions_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,0,2024-06-01 00:00:31,596,3156,57.16,31,0,0,0
1,1,2024-06-01 00:02:10,4961,3412,81.51,130,0,0,0
2,2,2024-06-01 00:07:56,2,1365,146.00,476,0,0,0
3,3,2024-06-01 00:09:29,4128,8737,64.49,569,0,0,0
4,4,2024-06-01 00:10:34,927,9906,50.99,634,0,0,0


## Date and time transformations

We will create two new binary features from the transaction dates and times:

* The first will characterize whether a transaction occurs during a weekday (value 0) or a weekend (1), and will be called `TX_DURING_WEEKEND`
* The second will characterize whether a transaction occurs during the day or during the day (0) or during the night (1). The night is defined as hours that are between 0pm and 6am. It will be called `TX_DURING_NIGHT`. 

For the `TX_DURING_WEEKEND` feature, we define a function `is_weekend` that takes as input a Panda timestamp, and returns 1 if the date is during a weekend, or 0 otherwise. The timestamp object conveniently provides the `weekday` function to help in computing this value.

In [42]:
def is_weekend(tx_datetime):
    
    # Transform date into weekday (0 is Monday, 6 is Sunday)
    weekday = tx_datetime.weekday()
    # Binary value: 0 if weekday, 1 if weekend
    is_weekend = weekday>=5
    
    return int(is_weekend)


It is then straghtforward to compute this feature for all transactions using the Panda `apply` function. 

In [43]:
%time transactions_df['TX_DURING_WEEKEND']=transactions_df.TX_DATETIME.apply(is_weekend)

CPU times: user 5.77 s, sys: 410 ms, total: 6.18 s
Wall time: 10.4 s


We follow the same logic to implement the `TX_DURING_NIGHT` feature. First, a function `is_night` that takes as input a Panda timestamp, and returns 1 if the time is during the night, or 0 otherwise. The timestamp object conveniently provides the hour property to help in computing this value.

In [44]:
def is_night(tx_datetime):
    
    # Get the hour of the transaction
    tx_hour = tx_datetime.hour
    # Binary value: 1 if hour less than 6, and 0 otherwise
    is_night = tx_hour<=6
    
    return int(is_night)

In [45]:
%time transactions_df['TX_DURING_NIGHT']=transactions_df.TX_DATETIME.apply(is_night)

CPU times: user 4.9 s, sys: 78.3 ms, total: 4.98 s
Wall time: 5 s


Let us check that these features where correctly computed.

In [46]:
transactions_df[transactions_df.TX_TIME_DAYS>=30]

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,TX_DURING_NIGHT
288062,288062,2024-07-01 00:01:21,3546,2944,18.71,2592081,30,0,0,0,1
288063,288063,2024-07-01 00:01:48,206,3521,18.60,2592108,30,0,0,0,1
288064,288064,2024-07-01 00:02:22,2610,4470,66.67,2592142,30,0,0,0,1
288065,288065,2024-07-01 00:03:15,4578,1520,79.41,2592195,30,0,0,0,1
288066,288066,2024-07-01 00:03:51,1246,7809,52.08,2592231,30,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
2051326,2051326,2024-12-31 23:54:48,664,7517,89.88,18489288,213,0,0,0,0
2051327,2051327,2024-12-31 23:55:05,377,4694,20.06,18489305,213,0,0,0,0
2051328,2051328,2024-12-31 23:58:10,2439,3969,26.18,18489490,213,0,0,0,0
2051329,2051329,2024-12-31 23:58:40,158,1733,80.90,18489520,213,0,0,0,0


## Customer ID transformations

Let us now proceed with customer ID transformations. We will take inspiration from the RFM (Recency, Frequency, Monetary value) framework proposed in {cite}`VANVLASSELAER201538`, and compute two of these features over three time windows. The first feature will be the number of transactions that occur within a time window (Frequency). The second will be the average amount spent in these transactions (Monetary value). The time windows will be set to one, seven, and thirty days. This will generate six new features. Note that these time windows could later be optimized along with the models using a model selection procedure. 

Let us implement these transformations by writing a `get_customer_spending_behaviour_features` function. The function takes as inputs the set of transactions for a customer and a set of window sizes. It returns a DataFrame with the six new features. Our implementation relies on the Panda `rolling` function, which makes easy the computation of aggregates over a time window.


In [47]:
def get_customer_spending_behaviour_features(customer_transactions, windows_size_in_days=[1,7,30]):
    
    # Let us first order transactions chronologically
    customer_transactions=customer_transactions.sort_values('TX_DATETIME')
    
    # The transaction date and time is set as the index, which will allow the use of the rolling function 
    customer_transactions.index=customer_transactions.TX_DATETIME
    
    # For each window size
    for window_size in windows_size_in_days:
        
        # Compute the sum of the transaction amounts and the number of transactions for the given window size
        SUM_AMOUNT_TX_WINDOW=customer_transactions['TX_AMOUNT'].rolling(str(window_size)+'d').sum()
        NB_TX_WINDOW=customer_transactions['TX_AMOUNT'].rolling(str(window_size)+'d').count()
    
        # Compute the average transaction amount for the given window size
        # NB_TX_WINDOW is always >0 since current transaction is always included
        AVG_AMOUNT_TX_WINDOW=SUM_AMOUNT_TX_WINDOW/NB_TX_WINDOW
    
        # Save feature values
        customer_transactions['CUSTOMER_ID_NB_TX_'+str(window_size)+'DAY_WINDOW']=list(NB_TX_WINDOW)
        customer_transactions['CUSTOMER_ID_AVG_AMOUNT_'+str(window_size)+'DAY_WINDOW']=list(AVG_AMOUNT_TX_WINDOW)
    
    # Reindex according to transaction IDs
    customer_transactions.index=customer_transactions.TRANSACTION_ID
        
    # And return the dataframe with the new features
    return customer_transactions


Let us compute these aggregates for the first customer.

In [48]:
spending_behaviour_customer_0=get_customer_spending_behaviour_features(transactions_df[transactions_df.CUSTOMER_ID==0])
spending_behaviour_customer_0

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,TX_DURING_NIGHT,CUSTOMER_ID_NB_TX_1DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW
TRANSACTION_ID,,,,,,,,,,,,,,,,,
1758,1758,2024-06-01 07:19:05,0,6076,123.59,26345,0,0,0,1,0,1.0,123.590000,1.0,123.590000,1.0,123.590000
8275,8275,2024-06-01 18:00:16,0,858,77.34,64816,0,0,0,1,0,2.0,100.465000,2.0,100.465000,2.0,100.465000
8640,8640,2024-06-01 19:02:02,0,6698,46.51,68522,0,0,0,1,0,3.0,82.480000,3.0,82.480000,3.0,82.480000
12169,12169,2024-06-02 08:51:06,0,6569,54.72,118266,1,0,0,1,0,3.0,59.523333,4.0,75.540000,4.0,75.540000
15764,15764,2024-06-02 14:05:38,0,7707,63.30,137138,1,0,0,1,0,4.0,60.467500,5.0,73.092000,5.0,73.092000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022803,2022803,2024-12-29 02:14:32,0,7618,65.77,18238472,211,0,0,1,1,4.0,47.542500,15.0,65.828000,83.0,60.039157
2024949,2024949,2024-12-29 08:21:29,0,330,64.36,18260489,211,0,0,1,0,4.0,42.267500,14.0,65.122857,84.0,60.090595
2043433,2043433,2024-12-31 06:59:44,0,1224,127.69,18428384,213,0,0,0,1,1.0,127.690000,12.0,71.460833,76.0,60.801447



Let us now generate these features for all customers. This is straightforward using the Panda `groupby` and `apply` methods.

In [49]:
%time transactions_df=transactions_df.groupby('CUSTOMER_ID').apply(lambda x: get_customer_spending_behaviour_features(x, windows_size_in_days=[1,7,30]))
transactions_df=transactions_df.sort_values('TX_DATETIME').reset_index(drop=True)


CPU times: user 26.3 s, sys: 316 ms, total: 26.6 s
Wall time: 28.4 s


In [50]:
transactions_df

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,TX_DURING_NIGHT,CUSTOMER_ID_NB_TX_1DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW
0,0,2024-06-01 00:00:31,596,3156,57.16,31,0,0,0,1,1,1.0,57.160000,1.0,57.160000,1.0,57.160000
1,1,2024-06-01 00:02:10,4961,3412,81.51,130,0,0,0,1,1,1.0,81.510000,1.0,81.510000,1.0,81.510000
2,2,2024-06-01 00:07:56,2,1365,146.00,476,0,0,0,1,1,1.0,146.000000,1.0,146.000000,1.0,146.000000
3,3,2024-06-01 00:09:29,4128,8737,64.49,569,0,0,0,1,1,1.0,64.490000,1.0,64.490000,1.0,64.490000
4,4,2024-06-01 00:10:34,927,9906,50.99,634,0,0,0,1,1,1.0,50.990000,1.0,50.990000,1.0,50.990000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2051326,2051326,2024-12-31 23:54:48,664,7517,89.88,18489288,213,0,0,0,0,6.0,90.418333,19.0,78.310526,81.0,95.328519
2051327,2051327,2024-12-31 23:55:05,377,4694,20.06,18489305,213,0,0,0,0,1.0,20.060000,3.0,11.750000,24.0,9.440417
2051328,2051328,2024-12-31 23:58:10,2439,3969,26.18,18489490,213,0,0,0,0,6.0,23.625000,24.0,19.128750,103.0,18.492816
2051329,2051329,2024-12-31 23:58:40,158,1733,80.90,18489520,213,0,0,0,0,5.0,85.688000,27.0,81.868889,137.0,85.895036


## Terminal ID transformations

Finally, let us proceed with the terminal ID transformations. The main goal will be to extract a *risk score*, that assesses the exposure of a given terminal ID to fraudulent transactions. The risk score will be defined as the average number of fraudulent transactions that occurred on a terminal ID over a time window. As for customer ID transformations, we will use three window sizes, of 1, 7, and 30 days.

Contrary to customer ID transformations, the time windows will not directly precede a given transaction. Instead, they will be shifted back by a *delay period*. The delay period accounts for the fact that, in practice, the fraudulent transactions are only discovered after a fraud investigation or a customer complaint. Hence, the fraudulent labels, which are needed to compute the risk score, are only available after this delay period. To a first approximation, this delay period will be set to one week. 

Let us perform the computation of the risk scores by defining a `get_count_risk_rolling_window` function. The function takes as inputs the DataFrame of transactions for a given terminal ID, the delay period, and a list of window sizes. In the first stage, the number of transactions and fraudulent transactions are computed for the delay period (`NB_TX_DELAY` and `NB_FRAUD_DELAY`). In the second stage, the number of transactions and fraudulent transactions are computed for each window size plus the delay period (`NB_TX_DELAY_WINDOW` and `NB_FRAUD_DELAY_WINDOW`). The number of transactions and fraudulent transactions that occurred for a given window size, shifted back by the delay period, is then obtained by simply computing the differences of the quantities obtained for the delay period, and the window size plus delay period:

```
NB_FRAUD_WINDOW=NB_FRAUD_DELAY_WINDOW-NB_FRAUD_DELAY
NB_TX_WINDOW=NB_TX_DELAY_WINDOW-NB_TX_DELAY
```

The risk score is finally obtained by computing the proportion of fraudulent transactions for each window size (or 0 if no transaction occurred for the given window):

```
RISK_WINDOW=NB_FRAUD_WINDOW/NB_TX_WINDOW
```

Additionally to the risk score, the function also returns the number of transactions for each window size. This results in the addition of six new features: The risk and number of transactions, for three window sizes.


In [51]:
def get_count_risk_rolling_window(terminal_transactions, delay_period=7, windows_size_in_days=[1,7,30], feature="TERMINAL_ID"):
    
    terminal_transactions=terminal_transactions.sort_values('TX_DATETIME')
    
    terminal_transactions.index=terminal_transactions.TX_DATETIME
    
    NB_FRAUD_DELAY=terminal_transactions['TX_FRAUD'].rolling(str(delay_period)+'d').sum()
    NB_TX_DELAY=terminal_transactions['TX_FRAUD'].rolling(str(delay_period)+'d').count()
    
    for window_size in windows_size_in_days:
    
        NB_FRAUD_DELAY_WINDOW=terminal_transactions['TX_FRAUD'].rolling(str(delay_period+window_size)+'d').sum()
        NB_TX_DELAY_WINDOW=terminal_transactions['TX_FRAUD'].rolling(str(delay_period+window_size)+'d').count()
    
        NB_FRAUD_WINDOW=NB_FRAUD_DELAY_WINDOW-NB_FRAUD_DELAY
        NB_TX_WINDOW=NB_TX_DELAY_WINDOW-NB_TX_DELAY
    
        RISK_WINDOW=NB_FRAUD_WINDOW/NB_TX_WINDOW
        
        terminal_transactions[feature+'_NB_TX_'+str(window_size)+'DAY_WINDOW']=list(NB_TX_WINDOW)
        terminal_transactions[feature+'_RISK_'+str(window_size)+'DAY_WINDOW']=list(RISK_WINDOW)
        
    terminal_transactions.index=terminal_transactions.TRANSACTION_ID
    
    # Replace NA values with 0 (all undefined risk scores where NB_TX_WINDOW is 0) 
    terminal_transactions.fillna(0,inplace=True)
    
    return terminal_transactions


In [52]:
transactions_df[transactions_df.TX_FRAUD==1]

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,TX_DURING_NIGHT,CUSTOMER_ID_NB_TX_1DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW
3527,3527,2024-06-01 10:17:43,3774,3059,225.41,37063,0,1,1,1,0,3.0,158.073333,3.0,158.073333,3.0,158.073333
5790,5790,2024-06-01 13:31:48,4944,6050,222.26,48708,0,1,1,1,0,2.0,127.605000,2.0,127.605000,2.0,127.605000
6549,6549,2024-06-01 14:42:02,4625,9102,226.40,52922,0,1,1,1,0,4.0,167.165000,4.0,167.165000,4.0,167.165000
9583,9583,2024-06-02 01:01:05,3814,6893,59.15,90065,1,1,3,1,1,6.0,29.138333,6.0,29.138333,6.0,29.138333
10355,10356,2024-06-02 05:03:35,2513,1143,222.04,104615,1,1,1,1,1,5.0,123.740000,5.0,123.740000,5.0,123.740000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2050670,2050670,2024-12-31 19:33:28,4220,4272,233.33,18473608,213,1,1,0,0,1.0,233.330000,3.0,155.530000,7.0,132.880000
2050679,2050679,2024-12-31 19:35:54,1018,9409,410.05,18473754,213,1,3,0,0,2.0,290.525000,11.0,236.919091,28.0,142.840714
2050692,2050692,2024-12-31 19:38:07,3931,8949,56.51,18473887,213,1,2,0,0,3.0,49.443333,16.0,46.781250,78.0,43.844231
2050926,2050926,2024-12-31 20:41:16,4669,6332,65.91,18477676,213,1,2,0,0,4.0,39.495000,8.0,44.000000,35.0,54.030571


Let us compute these six features for the first terminal ID containing at least one fraud:

In [53]:
# Get the first terminal ID that contains frauds
transactions_df[transactions_df.TX_FRAUD==0].TERMINAL_ID[0]

np.int64(3156)

In [54]:
get_count_risk_rolling_window(transactions_df[transactions_df.TERMINAL_ID==3059], delay_period=7, windows_size_in_days=[1,7,30])

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,...,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
TRANSACTION_ID,,,,,,,,,,,,,,,,,,,,,
3527,3527,2024-06-01 10:17:43,3774,3059,225.41,37063,0,1,1,1,...,3.0,158.073333,3.0,158.073333,0.0,0.0,0.0,0.0,0.0,0.0
4732,4732,2024-06-01 11:59:14,55,3059,36.28,43154,0,0,0,1,...,2.0,35.670000,2.0,35.670000,0.0,0.0,0.0,0.0,0.0,0.0
16216,16216,2024-06-02 14:47:34,4879,3059,105.00,139654,1,0,0,1,...,10.0,76.010000,10.0,76.010000,0.0,0.0,0.0,0.0,0.0,0.0
18249,18249,2024-06-02 19:08:10,2263,3059,90.89,155290,1,0,0,1,...,7.0,50.458571,7.0,50.458571,0.0,0.0,0.0,0.0,0.0,0.0
26512,26512,2024-06-03 15:44:49,4879,3059,58.51,229489,2,0,0,0,...,14.0,71.070000,14.0,71.070000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014871,2014871,2024-12-28 07:31:55,2045,3059,60.49,18171115,210,0,0,1,...,9.0,68.404444,57.0,65.574912,0.0,0.0,7.0,0.0,25.0,0.0
2015068,2015068,2024-12-28 07:54:42,1197,3059,35.20,18172482,210,0,0,1,...,19.0,67.722105,100.0,66.403600,0.0,0.0,6.0,0.0,25.0,0.0
2015647,2015647,2024-12-28 08:54:06,3377,3059,85.30,18176046,210,0,0,1,...,25.0,72.936400,105.0,70.518667,0.0,0.0,6.0,0.0,25.0,0.0


We can check that the first fraud occurred on the 2024/06/1 

Let us finally generate these features for all terminals. This is straightforward using the Panda `groupby` and `apply` methods. 

In [55]:
%time transactions_df=transactions_df.groupby('TERMINAL_ID').apply(lambda x: get_count_risk_rolling_window(x, delay_period=7, windows_size_in_days=[1,7,30], feature="TERMINAL_ID"))
transactions_df=transactions_df.sort_values('TX_DATETIME').reset_index(drop=True)


CPU times: user 1min, sys: 43.4 ms, total: 1min
Wall time: 1min


In [56]:
transactions_df

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,...,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
0,0,2024-06-01 00:00:31,596,3156,57.16,31,0,0,0,1,...,1.0,57.160000,1.0,57.160000,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2024-06-01 00:02:10,4961,3412,81.51,130,0,0,0,1,...,1.0,81.510000,1.0,81.510000,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2024-06-01 00:07:56,2,1365,146.00,476,0,0,0,1,...,1.0,146.000000,1.0,146.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2024-06-01 00:09:29,4128,8737,64.49,569,0,0,0,1,...,1.0,64.490000,1.0,64.490000,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2024-06-01 00:10:34,927,9906,50.99,634,0,0,0,1,...,1.0,50.990000,1.0,50.990000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2051326,2051326,2024-12-31 23:54:48,664,7517,89.88,18489288,213,0,0,0,...,19.0,78.310526,81.0,95.328519,0.0,0.0,5.0,0.0,29.0,0.0
2051327,2051327,2024-12-31 23:55:05,377,4694,20.06,18489305,213,0,0,0,...,3.0,11.750000,24.0,9.440417,2.0,0.0,6.0,0.0,19.0,0.0
2051328,2051328,2024-12-31 23:58:10,2439,3969,26.18,18489490,213,0,0,0,...,24.0,19.128750,103.0,18.492816,3.0,0.0,10.0,0.0,28.0,0.0
2051329,2051329,2024-12-31 23:58:40,158,1733,80.90,18489520,213,0,0,0,...,27.0,81.868889,137.0,85.895036,0.0,0.0,6.0,0.0,22.0,0.0


## Saving of dataset

Let us finally save the dataset, split into daily batches, using the pickle format. 

In [57]:
DIR_OUTPUT = "../data/processed/transaction/"

if not os.path.exists(DIR_OUTPUT):
    os.makedirs(DIR_OUTPUT)

start_date = datetime.datetime.strptime("2024-06-01", "%Y-%m-%d")

for day in range(transactions_df.TX_TIME_DAYS.max()+1):
    
    transactions_day = transactions_df[transactions_df.TX_TIME_DAYS==day].sort_values('TX_TIME_SECONDS')
    
    date = start_date + datetime.timedelta(days=day)
    filename_output = date.strftime("%Y-%m-%d")+'.pkl'
    
    
    transactions_day.to_pickle(DIR_OUTPUT+filename_output)

In [58]:
# In transactions_df, get the latest entry for each terminal id  basis tx_datetime
transactions_df_latest=transactions_df.loc[transactions_df.groupby('TERMINAL_ID').TX_DATETIME.idxmax()]

# select only temrinal id and risk features starting with TERMINAL_ID
transactions_df_latest=transactions_df_latest.filter(regex='TERMINAL_ID|TERMINAL_ID_RISK')


In [59]:
transactions_df_latest.head()



,TERMINAL_ID,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
2047792,0,1.0,0.0,6.0,0.0,36.0,0.0
2042135,1,1.0,0.0,6.0,0.0,21.0,0.0
2033552,2,1.0,0.0,4.0,0.0,30.0,0.0
2051286,3,1.0,0.0,7.0,0.0,21.0,0.0
2040707,4,2.0,0.0,10.0,0.0,29.0,0.0


In [60]:
# lower casee all column names
transactions_df_latest.columns = map(str.lower, transactions_df_latest.columns)

In [61]:
transactions_df_latest.head()

,terminal_id,terminal_id_nb_tx_1day_window,terminal_id_risk_1day_window,terminal_id_nb_tx_7day_window,terminal_id_risk_7day_window,terminal_id_nb_tx_30day_window,terminal_id_risk_30day_window
2047792,0,1.0,0.0,6.0,0.0,36.0,0.0
2042135,1,1.0,0.0,6.0,0.0,21.0,0.0
2033552,2,1.0,0.0,4.0,0.0,30.0,0.0
2051286,3,1.0,0.0,7.0,0.0,21.0,0.0
2040707,4,2.0,0.0,10.0,0.0,29.0,0.0


In [62]:
transactions_df_latest.columns

Index(['terminal_id', 'terminal_id_nb_tx_1day_window',
       'terminal_id_risk_1day_window', 'terminal_id_nb_tx_7day_window',
       'terminal_id_risk_7day_window', 'terminal_id_nb_tx_30day_window',
       'terminal_id_risk_30day_window'],
      dtype='object')

In [76]:
transactions_df_latest.to_pickle("../data/processed/feature_terminal.pkl")

In [64]:
DIR_INPUT='../data/raw/transaction/' 

BEGIN_DATE = "2025-01-01"
END_DATE = "2025-31-01"

print("Load  files")
%time transactions_df=read_from_files(DIR_INPUT, BEGIN_DATE, END_DATE)
print("{0} transactions loaded, containing {1} fraudulent transactions".format(len(transactions_df),transactions_df.TX_FRAUD.sum()))


Load  files
CPU times: user 609 ms, sys: 10.6 ms, total: 620 ms
Wall time: 677 ms
296988 transactions loaded, containing 2647 fraudulent transactions


In [65]:
%time transactions_df=transactions_df.groupby('CUSTOMER_ID').apply(lambda x: get_customer_spending_behaviour_features(x, windows_size_in_days=[1,7,30]))
transactions_df=transactions_df.sort_values('TX_DATETIME').reset_index(drop=True)


CPU times: user 39.2 s, sys: 368 ms, total: 39.6 s
Wall time: 39.9 s


In [66]:
transactions_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,CUSTOMER_ID_NB_TX_1DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW
0,2051331,2025-01-01 00:01:33,1959,9559,143.11,18489693,214,0,0,1.0,143.11,1.0,143.11,1.0,143.11
1,2051332,2025-01-01 00:02:26,658,9395,73.66,18489746,214,0,0,1.0,73.66,1.0,73.66,1.0,73.66
2,2051333,2025-01-01 00:03:36,4371,7721,99.55,18489816,214,0,0,1.0,99.55,1.0,99.55,1.0,99.55
3,2051334,2025-01-01 00:04:03,802,6022,50.71,18489843,214,0,0,1.0,50.71,1.0,50.71,1.0,50.71
4,2051335,2025-01-01 00:04:49,4630,2924,23.60,18489889,214,0,0,1.0,23.60,1.0,23.60,1.0,23.60


In [67]:
# lower casee all column names
transactions_df.columns = map(str.lower, transactions_df.columns)
customer_features=transactions_df.filter(regex='customer_id|tx_datetime')

In [68]:
customer_features.head()

,tx_datetime,customer_id,customer_id_nb_tx_1day_window,customer_id_avg_amount_1day_window,customer_id_nb_tx_7day_window,customer_id_avg_amount_7day_window,customer_id_nb_tx_30day_window,customer_id_avg_amount_30day_window
0,2025-01-01 00:01:33,1959,1.0,143.11,1.0,143.11,1.0,143.11
1,2025-01-01 00:02:26,658,1.0,73.66,1.0,73.66,1.0,73.66
2,2025-01-01 00:03:36,4371,1.0,99.55,1.0,99.55,1.0,99.55
3,2025-01-01 00:04:03,802,1.0,50.71,1.0,50.71,1.0,50.71
4,2025-01-01 00:04:49,4630,1.0,23.60,1.0,23.60,1.0,23.60


In [69]:
# filter where date(tx_datetime) is 2025-01-31
customer_features=customer_features[customer_features.tx_datetime.dt.date==datetime.date(2025,1,31)]

In [70]:
# convert to date
customer_features['dt']=customer_features['tx_datetime'].dt.date

In [71]:
# drop tx_datetime
customer_features.drop(columns=['tx_datetime'], inplace=True)

In [72]:
customer_features.to_pickle("../data/processed/feature_customer.pkl")

In [73]:
customer_data = {
    'customer_id': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],}
terminal_data = {
    'terminal_id': [0, 1, 2, 3, 4],
}

# filter customer and terminal features from customer_features and transactions_df_latest where customer id is in customer_data and terminal id is in terminal_data
customer_features=customer_features[customer_features.customer_id.isin(customer_data['customer_id'])]
transactions_df_latest=transactions_df_latest[transactions_df_latest.terminal_id.isin(terminal_data['terminal_id'])]


In [ ]:
customer_features



,customer_id,customer_id_nb_tx_1day_window,customer_id_avg_amount_1day_window,customer_id_nb_tx_7day_window,customer_id_avg_amount_7day_window,customer_id_nb_tx_30day_window,customer_id_avg_amount_30day_window,dt
287738,7,2.0,16.890000,20.0,31.836500,100.0,31.136700,2025-01-31
288420,1,4.0,61.860000,21.0,49.880000,110.0,48.530091,2025-01-31
288590,0,1.0,24.170000,20.0,67.519500,72.0,62.428750,2025-01-31
289312,4,3.0,63.703333,20.0,81.627000,108.0,78.478056,2025-01-31
289400,4,4.0,67.577500,21.0,81.511429,109.0,78.484679,2025-01-31
289552,4,5.0,69.222000,22.0,81.251818,110.0,78.460273,2025-01-31
289684,5,3.0,45.053333,20.0,49.279500,78.0,48.852436,2025-01-31
289730,1,5.0,57.040000,22.0,49.329091,110.0,48.091273,2025-01-31
291215,8,1.0,2.000000,13.0,6.510769,59.0,7.418136,2025-01-31
291653,9,2.0,82.690000,15.0,107.400000,80.0,98.033250,2025-01-31


In [75]:
transactions_df_latest

,terminal_id,terminal_id_nb_tx_1day_window,terminal_id_risk_1day_window,terminal_id_nb_tx_7day_window,terminal_id_risk_7day_window,terminal_id_nb_tx_30day_window,terminal_id_risk_30day_window
2047792,0,1.0,0.0,6.0,0.0,36.0,0.0
2042135,1,1.0,0.0,6.0,0.0,21.0,0.0
2033552,2,1.0,0.0,4.0,0.0,30.0,0.0
2051286,3,1.0,0.0,7.0,0.0,21.0,0.0
2040707,4,2.0,0.0,10.0,0.0,29.0,0.0


In [77]:
# There multiple entries for customer id in customer_features  deduplicate by customer id
customer_features=customer_features.drop_duplicates(subset=['customer_id'])

In [78]:
customer_features

,customer_id,customer_id_nb_tx_1day_window,customer_id_avg_amount_1day_window,customer_id_nb_tx_7day_window,customer_id_avg_amount_7day_window,customer_id_nb_tx_30day_window,customer_id_avg_amount_30day_window,dt
287738,7,2.0,16.890000,20.0,31.836500,100.0,31.136700,2025-01-31
288420,1,4.0,61.860000,21.0,49.880000,110.0,48.530091,2025-01-31
288590,0,1.0,24.170000,20.0,67.519500,72.0,62.428750,2025-01-31
289312,4,3.0,63.703333,20.0,81.627000,108.0,78.478056,2025-01-31
289684,5,3.0,45.053333,20.0,49.279500,78.0,48.852436,2025-01-31
291215,8,1.0,2.000000,13.0,6.510769,59.0,7.418136,2025-01-31
291653,9,2.0,82.690000,15.0,107.400000,80.0,98.033250,2025-01-31
292157,6,2.0,21.145000,21.0,18.560000,116.0,18.675259,2025-01-31
293266,3,1.0,16.010000,1.0,16.010000,12.0,12.614167,2025-01-31


In [79]:
customer_features.to_pickle("../data/processed/feature_customer.pkl")